In [11]:
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw/train.csv")    
OUT = Path("../data/processed/titanic_clean.csv")
OUT.parent.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(RAW)
print("Loaded:", RAW, "| shape:", df.shape)
df.head()


Loaded: ..\data\raw\train.csv | shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
print("Shape:", df.shape)
df.info()
df.describe()


Shape: (891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
df.isna().sum().sort_values(ascending=False)


Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
Pclass           0
Survived         0
Sex              0
Parch            0
SibSp            0
Fare             0
Ticket           0
dtype: int64

In [14]:
before = len(df)
df = df.drop_duplicates()
print("Removed duplicates:", before - len(df))

drop_cols = [c for c in ["Cabin","Ticket"] if c in df.columns] 
df = df.drop(columns=drop_cols)
print("Dropped:", drop_cols)


Removed duplicates: 0
Dropped: ['Cabin', 'Ticket']


In [15]:
if "Embarked" in df.columns and df["Embarked"].isna().any():
    df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

group_keys = [c for c in ["Sex","Pclass"] if c in df.columns]

# Age
if "Age" in df.columns and df["Age"].isna().any():
    if group_keys:
        df["Age"] = df.groupby(group_keys)["Age"].transform(lambda s: s.fillna(s.median()))
    df["Age"] = df["Age"].fillna(df["Age"].median())

# Fare
if "Fare" in df.columns and df["Fare"].isna().any():
    if group_keys:
        df["Fare"] = df.groupby(group_keys)["Fare"].transform(lambda s: s.fillna(s.median()))
    df["Fare"] = df["Fare"].fillna(df["Fare"].median())

df.isna().sum().sort_values(ascending=False).head(10)


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [16]:
for c in ["Sex","Embarked"]:
    if c in df.columns:
        df[c] = df[c].astype("category")

if "Pclass" in df.columns:
    df["Pclass"] = df["Pclass"].astype("category")

if "Survived" in df.columns:
    df["Survived"] = df["Survived"].astype(int)


In [17]:
def iqr_cap(s, k=3.0):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k*iqr, q3 + k*iqr
    return s.clip(lower, upper)

if "Fare" in df.columns:
    df["Fare"] = iqr_cap(df["Fare"], k=3.0)

df.describe()


,PassengerId,Survived,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,29.112424,0.523008,0.381594,27.043158
std,257.353842,0.486592,13.304424,1.102743,0.806057,27.339465
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,21.500000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,26.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,36.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,100.268800


In [18]:
# Family size
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

# IsAlone
df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

# Title extraction
df["Title"] = df["Name"].str.extract(r' ([A-Za-z]+)\.', expand=False)
df["Title"] = df["Title"].replace(
    ["Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"], "Rare"
)
df["Title"] = df["Title"].replace({"Mlle":"Miss", "Ms":"Miss", "Mme":"Mrs"})
df["Title"] = df["Title"].astype("category")

# Age bins
df["AgeBin"] = pd.cut(
    df["Age"],
    bins=[0, 12, 20, 40, 60, 80],
    labels=["Child", "Teen", "Adult", "MidAge", "Senior"]
)

# Fare bins (quartiles)
df["FareBin"] = pd.qcut(df["Fare"], 4, labels=["Q1", "Q2", "Q3", "Q4"])

df[["FamilySize", "IsAlone", "Title", "AgeBin", "FareBin"]].head(10)


,FamilySize,IsAlone,Title,AgeBin,FareBin
0,2,0,Mr,Adult,Q1
1,2,0,Mrs,Adult,Q4
2,1,1,Miss,Adult,Q2
3,2,0,Mrs,Adult,Q4
4,1,1,Mr,Adult,Q2
5,1,1,Mr,Adult,Q2
6,1,1,Mr,MidAge,Q4
7,5,0,Master,Child,Q3
8,3,0,Mrs,Adult,Q2
9,2,0,Mrs,Teen,Q3


In [19]:
df.to_csv(OUT, index=False)
print("Saved:", OUT, "| final shape:", df.shape)


Saved: ..\data\processed\titanic_clean.csv | final shape: (891, 15)
